<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/basisswapratehelpers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

Helper Functions: The core logic from the C++ testIborIborBootstrap and testOvernightIborBootstrap functions is moved into Python helper methods (_test_ibor_ibor_bootstrap, _test_overnight_ibor_bootstrap) within the test class to avoid duplication.
Data Representation: The BasisSwapQuote struct data is stored as a list of dictionaries for easier access within the Python loops.
Handles: ql.YieldTermStructureHandle and ql.RelinkableYieldTermStructureHandle are used to manage the curves.
Curve Bootstrapping: ql.PiecewiseLinearZero is used to perform linear interpolation on zero rates, matching the C++ PiecewiseYieldCurve<ZeroYield, Linear>. The settlementDays (or 0 for reference date) is passed as the first argument.
Index Linking: Critically, after bootstrapping, the correct index (baseIndex or otherIndex) is recreated or linked to the ql.YieldTermStructureHandle(bootstrappedCurve). This ensures the subsequent swap pricing uses the result of the bootstrap.
Discount Curve Handling (Overnight-IBOR): The externalDiscountCurve flag correctly determines whether the discountCurveHandle passed to the helpers is linked beforehand and whether the DiscountingSwapEngine uses this external curve or the newly bootstrapped curve for discounting.
Leg Creation: ql.IborLeg and ql.OvernightLeg are used. Note that the basis spread is added to the appropriate leg using .withSpreads().
Assertions: self.assertAlmostEqual checks if the NPV of the re-created swaps is close to zero, validating the bootstrap.

In [ ]:
import QuantLib as ql
import unittest

class BasisSwapRateHelpersTests(unittest.TestCase):

    def setUp(self):
        # Set evaluation date for tests
        self.today = ql.Date(11, ql.April, 2018) # Use a fixed date for consistency
        ql.Settings.instance().evaluationDate = self.today
        self.calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
        self.dayCounter = ql.Actual365Fixed()

    def tearDown(self):
        # Reset eval date after each test
        ql.Settings.instance().evaluationDate = ql.Date()

    def _test_ibor_ibor_bootstrap(self, bootstrapBaseCurve):
        """Helper function to test Ibor-Ibor bootstrap."""

        # Using a list of dictionaries for clarity
        quotes_data = [
            {'n': 1, 'units': ql.Years, 'basis': 0.0010},
            {'n': 2, 'units': ql.Years, 'basis': 0.0012},
            {'n': 3, 'units': ql.Years, 'basis': 0.0015},
            {'n': 5, 'units': ql.Years, 'basis': 0.0015},
            {'n': 8, 'units': ql.Years, 'basis': 0.0018},
            {'n': 10, 'units': ql.Years, 'basis': 0.0020},
            {'n': 15, 'units': ql.Years, 'basis': 0.0021},
            {'n': 20, 'units': ql.Years, 'basis': 0.0021},
        ]

        settlementDays = 2
        convention = ql.Following
        endOfMonth = False

        # Use Relinkable Handles for flexibility if needed, though FlatForward is simpler here
        knownForecastCurve = ql.YieldTermStructureHandle(ql.FlatForward(self.today, 0.01, self.dayCounter))
        discountCurve = ql.YieldTermStructureHandle(ql.FlatForward(self.today, 0.005, self.dayCounter))

        baseIndex = None
        otherIndex = None

        if bootstrapBaseCurve:
            # We bootstrap the curve for the 3M Libor index
            baseIndex = ql.USDLibor(ql.Period(3, ql.Months)) # No curve initially
            otherIndex = ql.USDLibor(ql.Period(6, ql.Months), knownForecastCurve)
        else:
            # We bootstrap the curve for the 6M Libor index
            baseIndex = ql.USDLibor(ql.Period(3, ql.Months), knownForecastCurve)
            otherIndex = ql.USDLibor(ql.Period(6, ql.Months)) # No curve initially

        helpers = []
        for q in quotes_data:
            basis_quote = ql.QuoteHandle(ql.SimpleQuote(q['basis']))
            tenor = ql.Period(q['n'], q['units'])
            # Constructor: basisHandle, tenor, settlementDays, calendar, convention,
            #              endOfMonth, baseIndex, quoteIndex, discountHandle, bootstrapBaseCurve=false
            h = ql.IborIborBasisSwapRateHelper(
                    basis_quote, tenor, settlementDays, self.calendar, convention, endOfMonth,
                    baseIndex, otherIndex, discountCurve, bootstrapBaseCurve)
            helpers.append(h)

        # Bootstrap the curve - use PiecewiseLinearZero for linear interp on zero rates
        bootstrappedCurve = ql.PiecewiseLinearZero(
            settlementDays, self.calendar, helpers, self.dayCounter)

        # Link the bootstrapped curve to the appropriate index
        bootstrappedCurveHandle = ql.YieldTermStructureHandle(bootstrappedCurve)
        if bootstrapBaseCurve:
            baseIndex = ql.USDLibor(ql.Period(3, ql.Months), bootstrappedCurveHandle)
            # otherIndex remains linked to knownForecastCurve
        else:
            # baseIndex remains linked to knownForecastCurve
            otherIndex = ql.USDLibor(ql.Period(6, ql.Months), bootstrappedCurveHandle)

        spot_date = self.calendar.advance(self.today, settlementDays, ql.Days)
        tolerance = 1.0e-8

        for q in quotes_data:
            # Create swaps and check they're fair
            maturity = self.calendar.advance(spot_date, q['n'], q['units'], convention)

            # Leg 1 (Base Index Leg - receives basis)
            s1 = ql.Schedule(spot_date, maturity, baseIndex.tenor(),
                             self.calendar, convention, convention, # Adjusted conventions to match C++ IborLeg default? No, should be from helper.
                             ql.DateGeneration.Forward, endOfMonth)
            leg1 = ql.IborLeg([100.0], s1, baseIndex)
            leg1 = leg1.withSpreads([q['basis']]) # Add basis spread

            # Leg 2 (Other Index Leg)
            s2 = ql.Schedule(spot_date, maturity, otherIndex.tenor(),
                             self.calendar, convention, convention,
                             ql.DateGeneration.Forward, endOfMonth)
            leg2 = ql.IborLeg([100.0], s2, otherIndex)

            # Swap pays leg1, receives leg2 (if basis is on leg1)
            swap = ql.Swap(leg1, leg2)
            swap.setPricingEngine(ql.DiscountingSwapEngine(discountCurve))

            NPV = swap.NPV()
            self.assertAlmostEqual(NPV, 0.0, delta=tolerance,
                                   msg=(f"Failed to price fair {q['n']}-{q['units']} swap "
                                        f"(bootstrapBaseCurve={bootstrapBaseCurve}): NPV = {NPV:.4e}"))


    def _test_overnight_ibor_bootstrap(self, externalDiscountCurve):
        """Helper function to test Overnight-Ibor bootstrap."""

        quotes_data = [
            {'n': 1, 'units': ql.Years, 'basis': 0.0010},
            {'n': 2, 'units': ql.Years, 'basis': 0.0012},
            {'n': 3, 'units': ql.Years, 'basis': 0.0015},
            {'n': 5, 'units': ql.Years, 'basis': 0.0015},
            {'n': 8, 'units': ql.Years, 'basis': 0.0018},
            {'n': 10, 'units': ql.Years, 'basis': 0.0020},
            {'n': 15, 'units': ql.Years, 'basis': 0.0021},
            {'n': 20, 'units': ql.Years, 'basis': 0.0021},
        ]

        settlementDays = 2
        convention = ql.Following
        endOfMonth = False

        knownForecastCurve = ql.YieldTermStructureHandle(ql.FlatForward(self.today, 0.01, self.dayCounter))

        discountCurveHandle = ql.RelinkableYieldTermStructureHandle()
        if externalDiscountCurve:
            discountCurveHandle.linkTo(ql.FlatForward(self.today, 0.005, self.dayCounter))
        # If not externalDiscountCurve, the handle remains empty, and Piecewise will use it for bootstrapping

        # Base index is Overnight (e.g., SOFR), linked to its known forecasting curve
        baseIndex = ql.Sofr(knownForecastCurve)
        # Other index is IBOR (e.g., USDLibor 6M), this is the one whose curve we bootstrap
        otherIndex = ql.USDLibor(ql.Period(6, ql.Months)) # No curve initially

        helpers = []
        for q in quotes_data:
            basis_quote = ql.QuoteHandle(ql.SimpleQuote(q['basis']))
            tenor = ql.Period(q['n'], q['units'])
            # Constructor: basisHandle, tenor, settlementDays, calendar, convention,
            #              endOfMonth, overnightIndex, iborIndex, discountCurve=Handle()
            h = ql.OvernightIborBasisSwapRateHelper(
                    basis_quote, tenor, settlementDays, self.calendar, convention, endOfMonth,
                    baseIndex, otherIndex, discountCurveHandle) # Pass potentially empty handle
            helpers.append(h)

        # Bootstrap the curve (which will be the forecast curve for otherIndex, and also the discount curve if discountCurveHandle was empty)
        bootstrappedCurve = ql.PiecewiseLinearZero(
            settlementDays, self.calendar, helpers, self.dayCounter)
        bootstrappedCurveHandle = ql.YieldTermStructureHandle(bootstrappedCurve)

        spot_date = self.calendar.advance(self.today, settlementDays, ql.Days)
        tolerance = 1.0e-8

        # Link the bootstrapped curve to the IBOR index
        otherIndex = ql.USDLibor(ql.Period(6, ql.Months), bootstrappedCurveHandle)
        # baseIndex remains linked to knownForecastCurve

        # Define the discount curve to be used by the engine
        engineDiscountCurveHandle = None
        if externalDiscountCurve:
            engineDiscountCurveHandle = discountCurveHandle # Use the externally provided one
        else:
            engineDiscountCurveHandle = bootstrappedCurveHandle # Use the bootstrapped curve for discounting

        for q in quotes_data:
            maturity = self.calendar.advance(spot_date, q['n'], q['units'], convention)

            # Use the same schedule tenor for both legs as per C++ (otherIndex tenor)
            schedule = ql.Schedule(spot_date, maturity, otherIndex.tenor(),
                                   self.calendar, convention, convention,
                                   ql.DateGeneration.Forward, endOfMonth)

            # Leg 1 (Overnight Leg - receives basis)
            leg1 = ql.OvernightLeg([100.0], schedule, baseIndex)
            leg1 = leg1.withSpreads([q['basis']])

            # Leg 2 (IBOR Leg)
            leg2 = ql.IborLeg([100.0], schedule, otherIndex)

            # Swap pays leg1 (OIS + basis), receives leg2 (IBOR)
            swap = ql.Swap(leg1, leg2)
            swap.setPricingEngine(ql.DiscountingSwapEngine(engineDiscountCurveHandle))

            NPV = swap.NPV()
            self.assertAlmostEqual(NPV, 0.0, delta=tolerance,
                                   msg=(f"Failed to price fair {q['n']}-{q['units']} OIS-IBOR swap "
                                        f"(externalDiscount={externalDiscountCurve}): NPV = {NPV:.4e}"))

    # --- Test Cases ---

    def test_ibor_ibor_base_curve_bootstrap(self):
        print("Testing IBOR-IBOR basis-swap rate helpers (base curve bootstrap)...")
        self._test_ibor_ibor_bootstrap(True)

    def test_ibor_ibor_other_curve_bootstrap(self):
        print("Testing IBOR-IBOR basis-swap rate helpers (other curve bootstrap)...")
        self._test_ibor_ibor_bootstrap(False)

    def test_overnight_ibor_bootstrap_without_discount_curve(self):
        print("Testing overnight-IBOR basis-swap rate helpers...")
        self._test_overnight_ibor_bootstrap(False)

    def test_overnight_ibor_bootstrap_with_discount_curve(self):
        print("Testing overnight-IBOR basis-swap rate helpers with external discount curve...")
        self._test_overnight_ibor_bootstrap(True)

if __name__ == '__main__':
    print("Python QuantLib version:", ql.__version__)
    print("Testing Basis Swap Rate Helpers (Python)...")
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(BasisSwapRateHelpersTests))
    unittest.TextTestRunner(verbosity=2).run(suite)